In [17]:
%pip install pyht

  Obtaining dependency information for pyht from https://files.pythonhosted.org/packages/ea/ea/1f57b0c4e076d26eb4ab09af4f6b65f1d39d48b6b1cfa14795fd905ab568/pyht-0.1.12-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.10.11 from https://files.pythonhosted.org/packages/a0/d6/2088fb4fd1e3ac2bfb24bc172223babaa7cdbb2784d33c75ec09e66f62f8/aiohttp-3.11.12-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for filelock<4.0.0,>=3.12.4 from https://files.pythonhosted.org/packages/89/ec/00d68c4ddfedfe64159999e5f8a98fb8442729a63e2077eb9dcd89623d27/filelock-3.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for grpcio<2.0.0,>=1.70.0 from https://files.pythonhosted.org/packages/e4/bd/cc36811c582d663a740fb45edf9f99ddbd99a10b6ba38267dc925e1e193a/grpcio-1.70.0-cp311-cp311-macosx_10_14_universal2.whl.metadata
  Obtaining dependency information for protobuf<6.0.0,>=5.29.3 from https://files.pythonhosted.org/packages/dd/04/3eaed

In [18]:
from lumaai import AsyncLumaAI, LumaAI
from dotenv import load_dotenv

import requests
import time
import os

from pyht import Client
from pyht.client import TTSOptions

load_dotenv()
client = LumaAI()

## Test Clip Descriptions from The Windup Girl

In [19]:
trailer_clips = [
    {
        "visual_description": "The camera swoops over a futuristic and industrial Bangkok submerged under an orange, polluted sky. Levees protect the city from swirling ocean waters.",
        "timing": "0-10 seconds",
        "audio_description": "Distant rumble of machines and crashing waves are heard.",
        "narrative": "In the 23rd century, a transformed world teeters on the brink of collapse."
    },
    {
        "visual_description": "Close-up of Anderson Lake in a crowded Bangkok street, his face shadowed, following a mysterious lead on the location of the Thai seedbank.",
        "timing": "11-20 seconds",
        "audio_description": "Tense strings begin to play, building suspense. Anderson's voiceover: \"In this world, seeds hold the key to power.\"",
        "narrative": "An agent of powerful forces, he seeks a treasure to control the destiny of nations."
    },
    {
        "visual_description": "In a vibrant, yet seedy club, Emiko, the 'Windup Girl', performs for a mesmerized audience, her movements fluid yet unnervingly precise.",
        "timing": "21-30 seconds",
        "audio_description": "Ambient club sounds mingle with haunting vocals. Emiko's whisper: \"There is a place for us, where we can be free.\"",
        "narrative": "A forbidden creation, she dreams of escape and redemption."
    },
    {
        "visual_description": "A flurry of chaos as the White Shirts, clad in distinctive uniforms, intercept illegal goods and clash with city forces.",
        "timing": "31-40 seconds",
        "audio_description": "Intense drumming and shouts echo through the scene. Snippets of rapid Thai dialogue punctuate the chaos.",
        "narrative": "A nation divided between control and chaos, teetering on the edge of civil war."
    },
    {
        "visual_description": "Quick cut to a shadowy meeting between Anderson and Akkarat, tension high as they negotiate around a dimly lit table.",
        "timing": "41-50 seconds",
        "audio_description": "The music quiets to a low, suspenseful hum. Anderson: \"I can give you what you desire... access to the seedbank.\"",
        "narrative": "Deals are struck in the shadows, as the fate of many hangs by a thread."
    },
    {
        "visual_description": "Emiko stands in the rain, trembling, eyes aflame with newfound determination after the brutal encounter at the club.",
        "timing": "51-60 seconds",
        "audio_description": "Thunder rolls as the music rises to a crescendo. Emiko's voiceover: \"I refuse to be a prisoner any longer.\"",
        "narrative": "From shattered dreams arise the seeds of rebellion."
    },
    {
        "visual_description": "Cut to Kanya, gazing at a vial of the new plague at the factory, her reflection distorted in the container.",
        "timing": "61-70 seconds",
        "audio_description": "A heartbeat bass underlines her internal conflict. Kanya whispers: \"I must stop this before it's too late.\"",
        "narrative": "In a race against time, one must confront their past to save the future."
    },
    {
        "visual_description": "The climactic destruction of Bangkok's levees shown in sweeping, dramatic sequences of water engulfing the city.",
        "timing": "71-80 seconds",
        "audio_description": "An epic orchestral score surges as the scene builds to its dramatic conclusion.",
        "narrative": "In chaos, new paths emerge, and destinies are rewritten."
    },
    {
        "visual_description": "Final scenes interweave shots of Emiko embraced by a community of 'New People'. Their hopeful faces stand out against the receding waters of a submerged Bangkok.",
        "timing": "81-90 seconds",
        "audio_description": "The music shifts to a hopeful, resonant theme. A voiceover: \"The world may change, but hope is eternal.\"",
        "narrative": "In the wreckage, a new beginning is born."
    }
]

# Test Generating the First 3 clips of the trailer

In [20]:
def text_to_speech(narrative_text, clip_number, book_title):
    client = Client(
        user_id=os.getenv("PLAY_HT_USER_ID"),
        api_key=os.getenv("PLAY_HT_API_KEY"),
    )
    options = TTSOptions(voice="s3://voice-cloning-zero-shot/775ae416-49bb-4fb6-bd45-740f205d20a1/jennifersaad/manifest.json")
    # Open a file to save the audio
    with open(f"first_clips/{book_title}/{clip_number}/narration.wav", "wb") as audio_file:
        for chunk in client.tts(narrative_text, options, voice_engine = 'PlayDialog-http'):
            # Write the audio chunk to the file
            audio_file.write(chunk)

In [21]:
book_title = "The Wind Up Girl"
first_sample = [{"video": t['visual_description'], "narration": t["narrative"]} for t in trailer_clips[:5]]

# create a directory in first_clips withthe book title and a subdirectory for each clip
try:
      os.makedirs(f'first_clips/{book_title}')
except FileExistsError:
      pass


# generate the first 5 clips
for i, clip in enumerate(first_sample):
   generation = client.generations.create(
   prompt=clip['video'],
   model='ray-2',
   resolution='540p',
   duration='5s',
   )

   completed = False
   while not completed:
      generation = client.generations.get(id=generation.id)
      if generation.state == "completed":
         completed = True
      elif generation.state == "failed":
         raise RuntimeError(f"Generation failed: {generation.failure_reason}")
      print("...")
      time.sleep(3)
   

   video_url = generation.assets.video
   response = requests.get(video_url, stream=True)
   # check to see if the directory exists
   video_path = f'first_clips/{book_title}/{i}'
   try:
      os.makedirs(video_path)
   except FileExistsError:
      pass
   with open(f'first_clips/{book_title}/{i}/{generation.id}.mp4', 'wb') as file:
      file.write(response.content)
   print(f"Video Clip downloaded as {generation.id}.mp4")
   
   text_to_speech(clip['narration'], i, book_title)
   print(f"Narration for clip {i} generated")


...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as 9ddc077c-e025-4ec6-ad15-b75507164128.mp4


Narration for clip 0 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as 6a533271-a7bf-44b6-9d00-6b613cb2022b.mp4


Narration for clip 1 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as 937dd9b0-1179-48fb-9771-026f1ee889dc.mp4


Narration for clip 2 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as fd0ec517-0ae3-40ac-af96-c1bae5594401.mp4


Narration for clip 3 generated
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
...
Video Clip downloaded as dfb79f23-98f2-4ceb-849e-5686e971ff6b.mp4


Narration for clip 4 generated


In [ ]:
generation = client.generations.create(
  prompt=first_sample,
  model='ray-2',
  resolution='540p',
  duration='5s',
)
completed = False
while not completed:
  generation = client.generations.get(id=generation.id)
  if generation.state == "completed":
    completed = True
  elif generation.state == "failed":
    raise RuntimeError(f"Generation failed: {generation.failure_reason}")
  print("Dreaming")
  time.sleep(3)

video_url = generation.assets.video

# download the video
response = requests.get(video_url, stream=True)
with open(f'first_clips/{generation.id}.mp4', 'wb') as file:
    file.write(response.content)
print(f"File downloaded as {generation.id}.mp4")
